# Calculate moisture flux component of AHT

By: Ty Janoski
Edited: 12.30.21

In [1]:
# import statements

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = "pdf"

In [2]:
# read in cell area file for taking spatial averages
areacella = xr.open_dataarray('/dx01/janoski/cesm/output/gridarea.nc')
# create function for taking spatial averages, while weighting for latitude
def spav(ds_in, areacella=areacella, lat_bound_s = -91, lat_bound_n = 91):
    """
    Use xarray/numpy to calculate spatial average while weighting for latitude.
    
    Keyword arguments:
    ds_in -- Dataset or DataArray to take the average of with coords lat and lon
    areacella -- Dataset or DataArray containing the grid cell area with coords lat and lon
    lat_bound_s -- float, Southern boundary of area to average
    lat_bound_n -- float, Northern boundary of area to average
    """
    ds_in = ds_in.sel(lat=slice(lat_bound_s,lat_bound_n))
    areacella = areacella.sel(lat=slice(lat_bound_s,lat_bound_n))
    out = (ds_in*(areacella/areacella.sum(dim=['lat','lon']))).sum(dim=['lat','lon'])
    return(out)

In [3]:
def read_in(exp,mon,ens,var):
    """
    Use xarray to read in a netCDF file.

    Keyword arguments:
    exp -- CO2 scenario
    mon -- starting month in which CO2 is altered
    ens -- ensemble number
    var -- model output variable
    """
    filein = '/dx01/janoski/cesm/output/b40.1850.cam5-lens.'+exp+'.'+str(
         f"{mon:02d}")+'.'+str(f"{ens:02d}")+'.h1_'+var+'.nc'
    return(xr.open_dataset(filein,chunks=None))

In [4]:
def read_in_AHT(exp,mon,ens,var):
    """
    Use xarray to read in a netCDF file.

    Keyword arguments:
    exp -- CO2 scenario
    mon -- starting month in which CO2 is altered
    ens -- ensemble number
    var -- model output variable
    """
    filein = '/dx02/janoski/cesm/ctrl_4xCO2_transports/b40.1850.cam5-lens.'+exp+'.'+str(
         f"{mon:02d}")+'.'+str(f"{ens:02d}")+'.h1_'+var+'.nc'
    return(xr.open_dataset(filein,chunks=None))

<b> Moisture flux equation: </b>

$\frac{dW}{dt} = E - P - \nabla\cdot\vec{W}$

We want:

$-\nabla\cdot\vec{W} = \frac{dW}{dt} - E + P $

In [5]:
# define some constants
ρ = 1000 # kg/m^3
Lv = 2.5e6 # J/kg
# m = 1
m=7

# iterate through each scenario
for s in ['ctrl','4xCO2']:
    # iterate through each ensemble member
    # for e in range(1,101,1):
    for e in range(1,51,1):
        print(e)
        PRECT = read_in(s,m,e,'PRECT').PRECT # m/s
        TMQ = read_in(s,m,e,'TMQ').TMQ # kg/m^2
        LHFLX = read_in(s,m,e,'LHFLX').LHFLX #W/m^2

        E = LHFLX # good as is
        P = PRECT * ρ * Lv # now in W/m^2
        dWdt = (TMQ * Lv).differentiate(coord='time',edge_order=2,datetime_unit='s') # now W/m^2

        # get convergence
        Wconv = (dWdt - E + P).rename('Wconv')
        del E, P, dWdt, PRECT, TMQ, LHFLX

        # read in AHT
        AHT = read_in_AHT(s,m,e,'Fconv_resi')

        # take diff to get DSE convergence
        Sconv = (AHT.AHT - Wconv).rename('Sconv')
        
        pathout = '/dx02/janoski/cesm/ctrl_4xCO2_transports/b40.1850.cam5-lens.'+s+'.'+str(f"{m:02d}")+'.'+str(f"{e:02d}")+'.h1_Wconv.nc'
        Wconv.to_netcdf(pathout)
        
        pathout = '/dx02/janoski/cesm/ctrl_4xCO2_transports/b40.1850.cam5-lens.'+s+'.'+str(f"{m:02d}")+'.'+str(f"{e:02d}")+'.h1_Sconv.nc'
        Sconv.to_netcdf(pathout)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
